In [1]:
import numpy as np
import pandas as pd

In [2]:
# Loading data
df = pd.read_csv('properties.csv')
df.head()

,Address,Suburb,State,Post code,Price,Property type,Bed,Bath,Parking
0,"3/9 Lambert Street,",CAMMERAY,NSW,2062,$690,Apartment / Unit / Flat,2 Beds,1 Bath,−
1,"18 Chatham Way,",HARRISDALE,WA,6112,$820 / PER WEEK,House,4 Beds,2 Baths,2 Parking
2,"102/157-163 Burwood Road,",HAWTHORN,VIC,3122,$580.00 per week,Apartment / Unit / Flat,2 Beds,1 Bath,1 Parking
3,"77 Unnaro St,",COOLOONGUP,WA,6168,$570 Per Week,House,3 Beds,2 Baths,3 Parking
4,"7/77 Milson Road,",CREMORNE POINT,NSW,2090,$850.00 pw,Apartment / Unit / Flat,2 Beds,1 Bath,−


### Explore the dataset

In [3]:
# Check for dimension
df.shape

(996, 9)

In [4]:
# Check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Address        996 non-null    object
 1   Suburb         996 non-null    object
 2   State          996 non-null    object
 3   Post code      996 non-null    int64 
 4   Price          996 non-null    object
 5   Property type  996 non-null    object
 6   Bed            996 non-null    object
 7   Bath           996 non-null    object
 8   Parking        996 non-null    object
dtypes: int64(1), object(8)
memory usage: 70.2+ KB


### Data cleaning

In [5]:
# Remove comma and any whitespace at the end of column "Address"
df['Address'] = df['Address'].str.rstrip(', ')

In [6]:
unique_suburb = df["Suburb"].nunique()
print(f"Number of unique suburbs: {unique_suburb}")

Number of unique suburbs: 475


In [7]:
df["Suburb"].unique()

array(['CAMMERAY', 'HARRISDALE', 'HAWTHORN', 'COOLOONGUP',
       'CREMORNE POINT', 'EARLWOOD', 'ST KILDA', 'MERRYLANDS', 'CAMILLO',
       'ROSEVILLE', 'GORDON', 'ULTIMO', 'ZETLAND', 'BANKSTOWN',
       'CAMPBELL', 'REDLYNCH', 'MOOROOBOOL', 'GREENSBOROUGH',
       'HELENSVALE', 'REDFERN', 'BRUNSWICK', 'EDGECLIFF', 'MAROUBRA',
       'SOUTH YARRA', 'NORTH MELBOURNE', 'TUART HILL', 'NORTH SYDNEY',
       'GREENWAY', 'KENSINGTON', 'SOUTHBANK', 'ASHFIELD', 'KINGSLEY',
       'MALVERN EAST', 'DUBBO', 'WOOLLAHRA', 'PORT MELBOURNE',
       'DARLINGHURST', 'KINGSTON', 'GUNGAHLIN', 'GRIFFIN', 'APPLECROSS',
       'COMO', 'ELWOOD', 'STRATHFIELD', 'ANNANDALE', 'MIDDLETON',
       'DARWIN CITY', 'WENTWORTH POINT', 'MELBOURNE', 'BRIGHTON',
       'PHILLIP', 'GLEN IRIS', 'RIPPONLEA', 'BONDI JUNCTION',
       'ST LEONARDS', 'LANE COVE', 'ORMOND', 'AUCHENFLOWER', 'CALDERWOOD',
       'CREMORNE', 'BARTON', 'ABBOTSFORD', 'CHIPPENDALE', 'POTTS POINT',
       'DOUBLE BAY', 'PENRITH', 'COBURG NORTH', 'CLA

There is no whitespaces in `Suburb`

In [8]:
df['State'].unique()

array(['NSW', 'WA', 'VIC', 'ACT', 'QLD', 'SA', 'NT', 'TAS'], dtype=object)

In [9]:
# Handling column "Price"
import re

def extract_numbers(price):
    if isinstance(price, str): # Ensure dtype of 'Price' is string
        match = re.search(r'\d[\d,]*', price) # Apply regex
        return int(match.group().replace(',', '')) if match else None # Remove commas and convert to integer
    return None

## Apply to column 'Price'
df['Price'] = df['Price'].apply(extract_numbers)
print(df)

                      Address          Suburb State  Post code   Price  \
0          3/9 Lambert Street        CAMMERAY   NSW       2062   690.0   
1              18 Chatham Way      HARRISDALE    WA       6112   820.0   
2    102/157-163 Burwood Road        HAWTHORN   VIC       3122   580.0   
3                77 Unnaro St      COOLOONGUP    WA       6168   570.0   
4            7/77 Milson Road  CREMORNE POINT   NSW       2090   850.0   
..                        ...             ...   ...        ...     ...   
991          73/1 Kiara Close    NORTH SYDNEY   NSW       2060  1250.0   
992         13 Northend Close       SWAN VIEW    WA       6056   580.0   
993    18/2-6 Junction Street            RYDE   NSW       2112   680.0   
994      648A Willoughby Road      WILLOUGHBY   NSW       2068  1150.0   
995            40 Orania Cres       CALAMVALE   QLD       4116   650.0   

               Property type     Bed     Bath    Parking  
0    Apartment / Unit / Flat  2 Beds   1 Bath       

**Inconsistancies in the "Price" column**

'Price' column contains values in various format:
   
   - Plain numeric values (`$690`)
   
   - Decimals (`$850.00`)
   
   - Descriptive formats (`$750 pw`, `$750 PER WEEK`, `$750 Per week`, `$750 per week`, `$750 weekly`)

   - Incorrect typos (`$4000/w`)
   
   - Comma-separated currency format (`$1,150`)
   
   - Noisy entries (`$650 per week!! Fully Air-conditioned Property!`)

*Standardizing price*: The pattern `\d[\d]*` matches any sequence starting with a digit followed by digits or commas. It will stop matching at the first non-digit/non-comma character.
   + `\d`: match the first digit
    
   + `[\d,]*`: match digits and commas, keep matching as long as the characters are digits and commas
   

In [10]:
# Extract only number if column Bed, Bath, and Parking
def extract_count(count):
    if isinstance(count, str):
        numbers = ''.join(re.findall(r'\d+', count)) # Use regex to find
        return int(numbers) if numbers else None
    return None

# Apply to column Bed, Bath, and Parking
df['Bed'] = df['Bed'].apply(extract_count)
df['Bath'] = df['Bath'].apply(extract_count)
df['Parking'] = df['Parking'].apply(extract_count)

In [11]:
# Handle missing values and convert values to int
df['Bed'] = df['Bed'].fillna(0).astype(int)
df['Bath'] = df['Bath'].fillna(0).astype(int)
df['Parking'] = df['Parking'].fillna(0).astype(int)

In [12]:
# Check for null values
df.isnull().sum()

Address          0
Suburb           0
State            0
Post code        0
Price            2
Property type    0
Bed              0
Bath             0
Parking          0
dtype: int64

In [13]:
null_price = df.loc[df['Price'].isna()]
null_price

,Address,Suburb,State,Post code,Price,Property type,Bed,Bath,Parking
634,1/19 Henderson Road,QUEANBEYAN,NSW,2620,NaN,Townhouse,2,2,1
651,705/71 Macquarie Street,SYDNEY,NSW,2000,NaN,Apartment / Unit / Flat,1,1,1


In [14]:
# Drop rows with missing values
df.dropna(inplace=True)

In [15]:
# Check for rows having no bed, bath, and parking
imcomplete_listing = df.loc[(df['Bed'] == 0) & (df['Bath'] == 0) & (df['Parking'] == 0)]
imcomplete_listing

,Address,Suburb,State,Post code,Price,Property type,Bed,Bath,Parking
76,5/24 Eliza Street,CLAYFIELD,QLD,4011,600.0,Apartment / Unit / Flat,0,0,0
836,705/9 Union Street,NUNDAH,QLD,4012,520.0,Apartment / Unit / Flat,0,0,0


In [16]:
# Remove rows having no bed, bath, and parking
df = df[~((df['Bed'] == 0) & (df['Bath'] == 0) & (df['Parking'] == 0))]
df.shape

(992, 9)

In [18]:
df.describe()

,Post code,Price,Bed,Bath,Parking
count,992.000000,992.000000,992.000000,992.000000,992.000000
mean,2881.787298,830.648185,2.356855,1.518145,1.230847
std,1042.593018,465.794169,1.135463,0.650757,0.859526
min,800.000000,1.000000,0.000000,1.000000,0.000000
25%,2064.000000,580.000000,2.000000,1.000000,1.000000
50%,2611.000000,720.000000,2.000000,1.000000,1.000000
75%,3182.000000,900.000000,3.000000,2.000000,2.000000
max,7306.000000,6000.000000,7.000000,6.000000,7.000000


In [19]:
df.to_csv('cleaned_property_data.csv', index=False)